# Features engineering

In [30]:
# Libraries and parameters
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import math

In [46]:
# Import data
data = pd.read_csv("../raw_data/final_raw_data_merged.csv", index_col="Unnamed: 0")

In [47]:
# #Creating hitter stats up to at bat
data['hitter_previous_stats_szn'] = data.groupby("hitter_id")["y_target"].cumsum() / (data.groupby('hitter_id').cumcount() + 1)
data['rolling_1ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

data['hitter_previous_stats_szn_slug'] = data.groupby("hitter_id")["mc_target"].cumsum() / (data.groupby('hitter_id').cumcount() + 1)
data['rolling_1ab_slug'] = data.groupby("hitter_id")["mc_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3ab_slug'] = data.groupby("hitter_id")["mc_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10ab_slug'] = data.groupby("hitter_id")["mc_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [48]:
# Calculate stats of pitcher before he pitchs
data['pitcher_previous_stats_szn'] = data.groupby("pitcher_id")["y_target"].cumsum() / (data.groupby('pitcher_id').cumcount() + 1)
data['rolling_1pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

data['pitcher_previous_stats_szn_bases'] = data.groupby("pitcher_id")["mc_target"].cumsum() / (data.groupby('pitcher_id').cumcount() + 1)
data['rolling_1pitch_bases'] = data.groupby("pitcher_id")["mc_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3pitch_bases'] = data.groupby("pitcher_id")["mc_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10pitch_bases'] = data.groupby("pitcher_id")["mc_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [49]:
data.head()

,id,game_id,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,humidity,at_bat_end_time,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,play_outcome,mc_target,y_target,pitch_type_cat,pitch_class,day_night,home_team_id,away_team_id,attendance,venue_id,hitter_player_name,hitter_position,hitter_primary_position,pitcher_player_name,pitcher_primary_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,hitter_previous_stats_szn_slug,rolling_1ab_slug,rolling_3ab_slug,rolling_10ab_slug,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,pitcher_previous_stats_szn_bases,rolling_1pitch_bases,rolling_3pitch_bases,rolling_10pitch_bases
0,d43f7325-ca19-49ab-94f6-b92055c26dd5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,041632a9-afb2-4ec3-b1de-9b0bbe33ab64,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:11:07+00:00,94.4,4.0,0.0,0.0,walk,1,1,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Wade Jr., LaMonte",IF,1B,"Cole, Gerrit",SP,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
1,f7f6d34f-5e1e-4ca9-896a-1f026054cbf5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,e5bdeb0e-38fc-4d30-8127-43d0d5b2864d,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:13:06+00:00,86.8,5.0,4.0,0.0,SO,0,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Conforto, Michael",OF,RF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.500000,1.0,1.000000,1.000000,0.500000,1.0,1.000000,1.000000
2,faad7e49-b9a2-4359-85a4-695438b6885c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,45bca5ce-4514-4a91-9410-e201c7b0052d,R,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:14:55+00:00,89.8,5.0,9.0,1.0,SO,0,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Flores, Wilmer",IF,1B,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.333333,0.0,0.500000,0.500000,0.333333,0.0,0.500000,0.500000
3,8f33e404-d2c3-427e-8a98-96ea1945285e,f3f0ae8e-cb65-4e96-9530-c6f868738f09,43a9d631-5673-4059-9b25-d59290bc32c3,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:18:09+00:00,89.1,7.0,14.0,2.0,SO,0,0,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Pederson, Joc",OF,LF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.250000,0.0,0.333333,0.333333,0.250000,0.0,0.333333,0.333333
4,b2fbcf1d-862e-4102-bb4e-fbac3656031c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,d1c07b01-a86d-4c17-ac38-8217e364c2cf,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,38.0,32.0,2023-03-30 17:21:39+00:00,85.9,4.0,0.0,0.0,SO,0,0,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"LeMahieu, David",IF,3B,"Webb, Logan",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [50]:
#Create handed matchups
data["handed_matchup"] = data["hitter_hand"] + data["pitcher_hand"]
data["handed_matchup"] = data.handed_matchup.apply((lambda x: 0 if x[0] == x[1] else 1))
# data = data.drop(columns=["hitter_hand", "pitcher_hand"])

In [51]:
#total hitter and pitcher at bats prior to at bat
data["hitter_ab_count"] = data.groupby('hitter_id').cumcount()
data["pitcher_ab_count"] = data.groupby('pitcher_id').cumcount()
data["match_up_ab_count_delta"] = data["pitcher_ab_count"] - data["hitter_ab_count"] 

In [52]:
#Balls and strikes efficiencies, batters
data["reverse_pitch_class"] = data["pitch_class"].apply(lambda x: 0 if x == 1 else 1)

data["hitter_strikes_atb"] = (data.groupby("hitter_id")["reverse_pitch_class"].cumsum())
data["hitter_balls_atb"] = (data.groupby("hitter_id")["pitch_class"].cumsum())

data["hitter_strikes_only_results"] = data["y_target"] * data["reverse_pitch_class"]
data["hitter_balls_only_results"] = data["y_target"] * data["pitch_class"]

data["hitter_strikes_results"] = (data.groupby("hitter_id")["hitter_strikes_only_results"].cumsum())
data["hitter_balls_results"] = (data.groupby("hitter_id")["hitter_balls_only_results"].cumsum())

data["hitter_strikes_eff"] = data["hitter_strikes_results"] / data["hitter_strikes_atb"]
data["hitter_balls_eff"] = data["hitter_balls_results"] / data["hitter_balls_atb"]

#Balls and strikes spread pitchers

data["pitcher_strikes_spread"] = (data.groupby("pitcher_id")["reverse_pitch_class"].cumsum()) / (data.groupby("pitcher_id")["pitcher_id"].cumcount() + 1)
data["pitcher_balls_spread"] = (data.groupby("pitcher_id")["pitch_class"].cumsum()) / (data.groupby("pitcher_id")["pitcher_id"].cumcount() + 1)


#pitch speed stats
data["hitter_speeds_class1"] = data["pitch_speed_mph"] * data["y_target"]
data["hitter_success_speed"] = (data.groupby("hitter_id")["hitter_speeds_class1"].cumsum()) / (data.groupby("hitter_id")["y_target"].cumsum())

data["pitcher_speed"] = (data.groupby("pitcher_id")["pitch_speed_mph"].cumsum())/ (data.groupby("pitcher_id").cumcount())

#Pitch type stats
data["reverse_pitch_type_cat"] = data["pitch_type_cat"].apply(lambda x: 0 if x == 1 else 1)

data["hitter_fast_atb"] = (data.groupby("hitter_id")["reverse_pitch_type_cat"].cumsum())
data["hitter_offspeed_atb"] = (data.groupby("hitter_id")["pitch_type_cat"].cumsum())

data["hitter_fast_only_results"] = data["y_target"] * data["reverse_pitch_type_cat"]
data["hitter_offspeed_only_results"] = data["y_target"] * data["pitch_type_cat"]

data["hitter_fast_results"] = (data.groupby("hitter_id")["hitter_fast_only_results"].cumsum())
data["hitter_offspeed_results"] = (data.groupby("hitter_id")["hitter_offspeed_only_results"].cumsum())

data["hitter_fast_eff"] = data["hitter_fast_results"] / data["hitter_fast_atb"]
data["hitter_offspeed_eff"] = data["hitter_offspeed_results"] / data["hitter_offspeed_atb"]

#Balls and strikes spread pitchers

data["pitcher_fast_spread"] = (data.groupby("pitcher_id")["reverse_pitch_type_cat"].cumsum()) / (data.groupby("pitcher_id")["pitcher_id"].cumcount() + 1)
data["pitcher_offspeed_spread"] = (data.groupby("pitcher_id")["pitch_type_cat"].cumsum()) / (data.groupby("pitcher_id")["pitcher_id"].cumcount() + 1)

#dropping redundant columns to avoid multi-colinearity

data = data.drop(columns=["pitch_class", "reverse_pitch_class", "hitter_strikes_atb", "hitter_balls_atb", "hitter_strikes_only_results",
                         "hitter_balls_only_results", "hitter_strikes_results", "hitter_balls_results", "hitter_speeds_class1",
                         "reverse_pitch_type_cat", "hitter_fast_atb", "hitter_offspeed_atb", "hitter_fast_only_results","hitter_offspeed_only_results","hitter_fast_results", "hitter_offspeed_results"])

In [53]:
data.head()

,id,game_id,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,humidity,at_bat_end_time,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,play_outcome,mc_target,y_target,pitch_type_cat,day_night,home_team_id,away_team_id,attendance,venue_id,hitter_player_name,hitter_position,hitter_primary_position,pitcher_player_name,pitcher_primary_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,hitter_previous_stats_szn_slug,rolling_1ab_slug,rolling_3ab_slug,rolling_10ab_slug,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,pitcher_previous_stats_szn_bases,rolling_1pitch_bases,rolling_3pitch_bases,rolling_10pitch_bases,handed_matchup,hitter_ab_count,pitcher_ab_count,match_up_ab_count_delta,hitter_strikes_eff,hitter_balls_eff,pitcher_strikes_spread,pitcher_balls_spread,hitter_success_speed,pitcher_speed,hitter_fast_eff,hitter_offspeed_eff,pitcher_fast_spread,pitcher_offspeed_spread
0,d43f7325-ca19-49ab-94f6-b92055c26dd5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,041632a9-afb2-4ec3-b1de-9b0bbe33ab64,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:11:07+00:00,94.4,4.0,0.0,0.0,walk,1,1,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Wade Jr., LaMonte",IF,1B,"Cole, Gerrit",SP,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1,0,0,0,NaN,1.0,0.000000,1.000000,94.4,inf,1.0,NaN,1.0,0.0
1,f7f6d34f-5e1e-4ca9-896a-1f026054cbf5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,e5bdeb0e-38fc-4d30-8127-43d0d5b2864d,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:13:06+00:00,86.8,5.0,4.0,0.0,SO,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Conforto, Michael",OF,RF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.500000,1.0,1.000000,1.000000,0.500000,1.0,1.000000,1.000000,1,0,1,1,0.0,NaN,0.500000,0.500000,NaN,181.200000,0.0,NaN,1.0,0.0
2,faad7e49-b9a2-4359-85a4-695438b6885c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,45bca5ce-4514-4a91-9410-e201c7b0052d,R,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:14:55+00:00,89.8,5.0,9.0,1.0,SO,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Flores, Wilmer",IF,1B,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.333333,0.0,0.500000,0.500000,0.333333,0.0,0.500000,0.500000,0,0,2,2,0.0,NaN,0.666667,0.333333,NaN,135.500000,0.0,NaN,1.0,0.0
3,8f33e404-d2c3-427e-8a98-96ea1945285e,f3f0ae8e-cb65-4e96-9530-c6f868738f09,43a9d631-5673-4059-9b25-d59290bc32c3,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:18:09+00:00,89.1,7.0,14.0,2.0,SO,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Pederson, Joc",OF,LF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.250000,0.0,0.333333,0.333333,0.250000,0.0,0.333333,0.333333,1,0,3,3,NaN,0.0,0.500000,0.500000,NaN,120.033333,0.0,NaN,1.0,0.0
4,b2fbcf1d-862e-4102-bb4e-fbac3656031c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,d1c07b01-a86d-4c17-ac38-8217e364c2cf,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,38.0,32.0,2023-03-30 17:21:39+00:00,85.9,4.0,0.0,0.0,SO,0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"LeMahieu, David",IF,3B,"Webb, Logan",SP,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0,0,0,0,NaN,0.0,0.000000,1.000000,NaN,inf,0.0,NaN,1.0,0.0


In [54]:
#drop 0.004343% of data (first ten at bats for each player)
data = data.dropna(subset=["rolling_10ab", "rolling_10pitch", "hitter_strikes_eff", "hitter_balls_eff", "hitter_success_speed", "hitter_offspeed_eff", 'hitter_fast_eff'])

In [55]:
data.isnull().sum()

id                                    0
game_id                               0
hitter_id                             0
hitter_hand                           0
pitcher_id                            0
pitcher_hand                          0
temp_f                                0
humidity                              0
at_bat_end_time                       0
pitch_speed_mph                       0
pitch_count_at_bat                    0
pitcher_pitch_count_at_bat_start      0
outs_at_start                         0
play_outcome                          0
mc_target                             0
y_target                              0
pitch_type_cat                        0
day_night                             0
home_team_id                          0
away_team_id                          0
attendance                          438
venue_id                              0
hitter_player_name                    0
hitter_position                       0
hitter_primary_position               0


In [56]:
data.columns

Index(['id', 'game_id', 'hitter_id', 'hitter_hand', 'pitcher_id',
       'pitcher_hand', 'temp_f', 'humidity', 'at_bat_end_time',
       'pitch_speed_mph', 'pitch_count_at_bat',
       'pitcher_pitch_count_at_bat_start', 'outs_at_start', 'play_outcome',
       'mc_target', 'y_target', 'pitch_type_cat', 'day_night', 'home_team_id',
       'away_team_id', 'attendance', 'venue_id', 'hitter_player_name',
       'hitter_position', 'hitter_primary_position', 'pitcher_player_name',
       'pitcher_primary_position', 'hitter_previous_stats_szn', 'rolling_1ab',
       'rolling_3ab', 'rolling_10ab', 'hitter_previous_stats_szn_slug',
       'rolling_1ab_slug', 'rolling_3ab_slug', 'rolling_10ab_slug',
       'pitcher_previous_stats_szn', 'rolling_1pitch', 'rolling_3pitch',
       'rolling_10pitch', 'pitcher_previous_stats_szn_bases',
       'rolling_1pitch_bases', 'rolling_3pitch_bases', 'rolling_10pitch_bases',
       'handed_matchup', 'hitter_ab_count', 'pitcher_ab_count',
       'match_up_ab_co

In [57]:
#Creating final training set with only ket features
final_modeling_columns = ['hitter_position', 'hitter_primary_position', 'hitter_previous_stats_szn', 'rolling_1ab',
                    'rolling_3ab', 'rolling_10ab', 'hitter_previous_stats_szn_slug',
                    'rolling_1ab_slug', 'rolling_3ab_slug', 'rolling_10ab_slug',
                    'pitcher_previous_stats_szn', 'rolling_1pitch', 'rolling_3pitch',
                    'rolling_10pitch', 'pitcher_previous_stats_szn_bases',
                    'rolling_1pitch_bases', 'rolling_3pitch_bases', 'rolling_10pitch_bases',
                    'handed_matchup','match_up_ab_count_delta', 'hitter_strikes_eff', 'hitter_balls_eff',
                    'pitcher_strikes_spread', 'pitcher_balls_spread',
                    'hitter_success_speed', 'pitcher_speed', 'hitter_fast_eff',
                    'hitter_offspeed_eff', 'pitcher_fast_spread',
                    'pitcher_offspeed_spread', 'y_target']



full_set_columns = ['hitter_id', 'hitter_hand', 'pitcher_id', 'pitcher_hand', 'at_bat_end_time',
                        'hitter_position', 'hitter_previous_stats_szn', 'rolling_1ab',
                        'rolling_3ab', 'rolling_10ab', 'hitter_previous_stats_szn_slug',
                        'rolling_1ab_slug', 'rolling_3ab_slug', 'rolling_10ab_slug',
                        'pitcher_previous_stats_szn', 'rolling_1pitch', 'rolling_3pitch',
                        'rolling_10pitch', 'pitcher_previous_stats_szn_bases',
                        'rolling_1pitch_bases', 'rolling_3pitch_bases', 'rolling_10pitch_bases',
                        'handed_matchup','match_up_ab_count_delta', 'hitter_strikes_eff', 'hitter_balls_eff',
                        'pitcher_strikes_spread', 'pitcher_balls_spread',
                        'hitter_success_speed', 'pitcher_speed', 'hitter_fast_eff',
                        'hitter_offspeed_eff', 'pitcher_fast_spread',
                        'pitcher_offspeed_spread', 'y_target']

data_full = data[full_set_columns]
data = data[final_modeling_columns]

In [58]:
data.to_csv("../preproc_data/data_with_new_features.csv")
data_full.to_csv("../preproc_data/final_full_dataset.csv")

In [59]:
data_full

,hitter_id,hitter_hand,pitcher_id,pitcher_hand,at_bat_end_time,hitter_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,hitter_previous_stats_szn_slug,rolling_1ab_slug,rolling_3ab_slug,rolling_10ab_slug,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,pitcher_previous_stats_szn_bases,rolling_1pitch_bases,rolling_3pitch_bases,rolling_10pitch_bases,handed_matchup,match_up_ab_count_delta,hitter_strikes_eff,hitter_balls_eff,pitcher_strikes_spread,pitcher_balls_spread,hitter_success_speed,pitcher_speed,hitter_fast_eff,hitter_offspeed_eff,pitcher_fast_spread,pitcher_offspeed_spread,y_target
72,86f7390e-61bd-4556-8325-a6705c7f693b,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,2023-03-30 18:30:43+00:00,OF,0.333333,0.0,0.500000,0.5,1.333333,0.0,2.000000,2.0,0.250000,0.0,0.000000,0.200000,0.550000,0.0,0.000000,0.500000,0,17,0.500000,0.000000,0.600000,0.400000,92.800000,93.131579,0.000000,0.500000,0.300000,0.700000,0
93,0e0faed9-3d08-418c-b4dc-f2e993483a36,R,e10239f6-8a6e-42af-bfb1-03fdf173af3d,R,2023-03-30 18:41:16+00:00,IF,0.500000,1.0,1.000000,1.0,0.500000,1.0,1.000000,1.0,0.285714,0.0,0.000000,0.333333,0.285714,0.0,0.000000,0.333333,0,5,0.000000,1.000000,0.714286,0.285714,89.700000,104.266667,1.000000,0.000000,0.428571,0.571429,0
95,267c9cd7-a2e2-4f88-8b9b-b7389b1a9ccd,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,2023-03-30 18:42:38+00:00,IF,0.333333,1.0,0.500000,0.5,0.333333,1.0,0.500000,0.5,0.217391,0.0,0.000000,0.200000,0.478261,0.0,0.000000,0.500000,0,20,0.500000,0.000000,0.565217,0.434783,84.700000,92.345455,1.000000,0.000000,0.304348,0.695652,0
114,d4bc8879-c7dc-430f-bcb8-e932b21a2e7c,L,48982279-0e43-4d94-95fb-fd5e623b54b6,R,2023-03-30 18:54:11+00:00,OF,0.500000,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.416667,1.0,0.333333,0.400000,0.750000,1.0,0.333333,0.700000,1,10,1.000000,0.000000,0.583333,0.416667,86.100000,93.018182,0.000000,1.000000,0.166667,0.833333,1
129,0203fb2b-3d49-40bd-b685-c6a5c388cade,L,f6e5344f-c816-460f-87ab-9322cfdc08b3,R,2023-03-30 19:05:12+00:00,IF,0.666667,0.0,0.500000,0.5,0.666667,0.0,0.500000,0.5,0.500000,0.0,0.333333,0.333333,0.500000,0.0,0.333333,0.333333,1,1,0.500000,1.000000,0.500000,0.500000,89.050000,117.466667,0.000000,1.000000,0.250000,0.750000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142302,ca159e78-05a9-410a-be7b-3ebad5496a88,L,22dabc06-611e-4d36-81af-7153e7ba86e6,R,2023-08-23 04:24:16+00:00,IF,0.402145,1.0,0.333333,0.3,0.675603,1.0,0.333333,0.3,0.277228,0.0,0.000000,0.300000,0.405941,0.0,0.000000,0.300000,1,-171,0.382900,0.451923,0.663366,0.336634,87.950667,88.826368,0.385027,0.419355,0.529703,0.470297,1
142303,479ea4a2-43ea-4f5f-93be-b875aab7088d,L,22dabc06-611e-4d36-81af-7153e7ba86e6,R,2023-08-23 04:26:37+00:00,IF,0.366548,1.0,0.666667,0.3,0.516014,1.0,0.666667,0.4,0.280788,1.0,0.333333,0.300000,0.408867,1.0,0.333333,0.300000,1,-359,0.332487,0.446429,0.665025,0.334975,89.513592,88.842574,0.385294,0.337838,0.532020,0.467980,1
142304,8f44f889-278d-4321-9158-bb8a4a7ef271,R,22dabc06-611e-4d36-81af-7153e7ba86e6,R,2023-08-23 04:27:53+00:00,OF,0.329545,0.0,0.000000,0.1,0.551136,0.0,0.000000,0.4,0.284314,1.0,0.666667,0.300000,0.411765,1.0,0.666667,0.300000,0,-324,0.284830,0.400000,0.666667,0.333333,89.332759,88.860099,0.324242,0.338384,0.534314,0.465686,1
142305,a8efa694-2d64-4ad3-bf39-7d4f9006be7f,R,22dabc06-611e-4d36-81af-7153e7ba86e6,R,2023-08-23 04:30:56+00:00,C,0.355769,0.0,0.333333,0.2,0.548077,0.0,1.333333,0.5,0.282927,1.0,1.000000,0.400000,0.409756,1.0,1.000000,0.400000,0,-3,0.354610,0.358209,0.663415,0.336585,89.832432,88.841176,0.387097,0.309524,0.531707,0.468293,0


In [111]:
data

,temp_f,humidity,pitcher_pitch_count_at_bat_start,outs_at_start,hitter_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,hitter_previous_stats_szn_slug,rolling_1ab_slug,rolling_3ab_slug,rolling_10ab_slug,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,pitcher_previous_stats_szn_bases,rolling_1pitch_bases,rolling_3pitch_bases,rolling_10pitch_bases,handed_matchup,match_up_ab_count_delta,hitter_strikes_eff,hitter_balls_eff,pitcher_strikes_spread,pitcher_balls_spread,hitter_success_speed,pitcher_speed,hitter_fast_eff,hitter_offspeed_eff,pitcher_fast_spread,pitcher_offspeed_spread,y_target
72,42.0,26.0,70.0,2.0,OF,0.333333,0.0,0.500000,0.5,1.333333,0.0,2.000000,2.0,0.250000,0.0,0.000000,0.200000,0.550000,0.0,0.000000,0.500000,0,17,0.500000,0.000000,0.600000,0.400000,92.800000,93.131579,0.000000,0.500000,0.300000,0.700000,0
93,50.0,29.0,22.0,2.0,IF,0.500000,1.0,1.000000,1.0,0.500000,1.0,1.000000,1.0,0.285714,0.0,0.000000,0.333333,0.285714,0.0,0.000000,0.333333,0,5,0.000000,1.000000,0.714286,0.285714,89.700000,104.266667,1.000000,0.000000,0.428571,0.571429,0
95,42.0,26.0,81.0,2.0,IF,0.333333,1.0,0.500000,0.5,0.333333,1.0,0.500000,0.5,0.217391,0.0,0.000000,0.200000,0.478261,0.0,0.000000,0.500000,0,20,0.500000,0.000000,0.565217,0.434783,84.700000,92.345455,1.000000,0.000000,0.304348,0.695652,0
114,38.0,23.0,53.0,2.0,OF,0.500000,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.416667,1.0,0.333333,0.400000,0.750000,1.0,0.333333,0.700000,1,10,1.000000,0.000000,0.583333,0.416667,86.100000,93.018182,0.000000,1.000000,0.166667,0.833333,1
129,50.0,29.0,12.0,0.0,IF,0.666667,0.0,0.500000,0.5,0.666667,0.0,0.500000,0.5,0.500000,0.0,0.333333,0.333333,0.500000,0.0,0.333333,0.333333,1,1,0.500000,1.000000,0.500000,0.500000,89.050000,117.466667,0.000000,1.000000,0.250000,0.750000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142302,96.0,28.0,6.0,1.0,IF,0.402145,1.0,0.333333,0.3,0.675603,1.0,0.333333,0.3,0.277228,0.0,0.000000,0.300000,0.405941,0.0,0.000000,0.300000,1,-171,0.382900,0.451923,0.663366,0.336634,87.950667,88.826368,0.385027,0.419355,0.529703,0.470297,1
142303,96.0,28.0,12.0,1.0,IF,0.366548,1.0,0.666667,0.3,0.516014,1.0,0.666667,0.4,0.280788,1.0,0.333333,0.300000,0.408867,1.0,0.333333,0.300000,1,-359,0.332487,0.446429,0.665025,0.334975,89.513592,88.842574,0.385294,0.337838,0.532020,0.467980,1
142304,96.0,28.0,18.0,1.0,OF,0.329545,0.0,0.000000,0.1,0.551136,0.0,0.000000,0.4,0.284314,1.0,0.666667,0.300000,0.411765,1.0,0.666667,0.300000,0,-324,0.284830,0.400000,0.666667,0.333333,89.332759,88.860099,0.324242,0.338384,0.534314,0.465686,1
142305,96.0,28.0,20.0,1.0,C,0.355769,0.0,0.333333,0.2,0.548077,0.0,1.333333,0.5,0.282927,1.0,1.000000,0.400000,0.409756,1.0,1.000000,0.400000,0,-3,0.354610,0.358209,0.663415,0.336585,89.832432,88.841176,0.387097,0.309524,0.531707,0.468293,0
